In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.transforms import v2
import pickle
import hashlib
import random
import numpy as np
import cv2
from PIL import Image
import os
import zipfile
from tqdm import tqdm

In [ ]:
##########################################################################
#                               URL DO DATASET                           #
##########################################################################

# diretorio do dataset BALANCEADO
diretorio_destino = 'data/base_treinamento/'

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1jyx7SzixgxVWmXWB-5y1frZQ89q5CSvp
From (redirected): https://drive.google.com/uc?id=1jyx7SzixgxVWmXWB-5y1frZQ89q5CSvp&confirm=t&uuid=96fd9aa2-c20e-4891-be04-84a9b3999ed5
To: /content/id_base_recortada.zip
100% 159M/159M [00:00<00:00, 202MB/s]


### Geração dos centros pelo método Radomico Melhorado

In [ ]:
####################################
# Random Pixels Melhorado
####################################
def pixels_adj( matriz, x, y, n_voltas):
  coords_voltas = []
  n_linhas, n_colunas = matriz.shape

  for volta in range(1, n_voltas + 1):
      for i in range(-volta, volta + 1):
          if 0 <= y + i < n_linhas:
              if 0 <= x - volta < n_colunas:
                  coords_voltas.append((y + i, x - volta))
              if 0 <= x + volta < n_colunas:
                  coords_voltas.append((y + i, x + volta))

      for j in range(-volta + 1, volta):
          if 0 <= x + j < n_colunas:
              if 0 <= y - volta < n_linhas:
                  coords_voltas.append((y - volta, x + j))
              if 0 <= y + volta < n_linhas:
                  coords_voltas.append((y + volta, x + j))
  coords_voltas.append((y,x))
  return coords_voltas

def verificar_adj(matriz, x, y, lista_centros):
    x, y = int(round(x)), int(round(y))
    if len(lista_centros) == 0:
        return False
    else:
        for each in lista_centros:
            each_x, each_y = int(round(each[1])), int(round(each[0]))
            ret = pixels_adj(matriz=matriz, x=each_x, y=each_y, n_voltas=8)
            if (y, x) in ret:
                return True
    return False

def random_patchs_melhorados( patch_size, num_patches, imagem_tensor):

    img_PIL = Image.fromarray((imagem_tensor.permute(1, 2, 0).cpu().numpy() * 255).astype(np.uint8))

    image_height, image_width = img_PIL.size

    patch_height, patch_width = patch_size

    img_gray = img_PIL.convert('L')
    img_mtx = np.array(img_gray)

    centers = []

    for _ in range(num_patches):
        h = random.uniform(patch_height / 2, image_height - patch_height / 2)
        w = random.uniform(patch_width / 2, image_width - patch_width / 2)


        check = verificar_adj(img_mtx, w, h, centers)

        while check:
            h = random.uniform(patch_height / 2, image_height - patch_height / 2)
            w = random.uniform(patch_width / 2, image_width - patch_width / 2)
            check = verificar_adj(img_mtx, w, h, centers)

        centers.append((h, w))

    print(len(centers))

    return centers

In [ ]:
import os
from torchvision.datasets import ImageFolder

class CustomImageFolder(ImageFolder):
    def __getitem__(self, index):
        img, target = super().__getitem__(index)

        path = self.imgs[index][0]

        image_name = os.path.basename(path)

        return img, target, image_name

In [ ]:
pickle_file = "randomico_melhorado_identificador_por_imgname.pkl"

img_size = (224, 224)
batch_size = 32

transform = v2.Compose([
    v2.Resize(img_size),
    v2.ToTensor(),
])

dataset = CustomImageFolder(root=diretorio_destino, transform=transform)


print(f"Tamanho do dataset é de {len(dataset)} imagens")


# lista_centro_dict = []

centros_dict = {}

# Iteração sobre as imagens do dataset
# for img_idx in range(len(dataset)):
for img_idx in tqdm(range(len(dataset)), desc="Processando imagens"):
    image, label, image_name = dataset[img_idx]

    centers = random_patchs_melhorados( patch_size=(16,16), num_patches=196, imagem_tensor=image)


    centros_dict[image_name] = centers


# Salvando o dicionário em um arquivo pickle
with open(pickle_file, 'wb') as f:
    pickle.dump(centros_dict, f)


print(f"Centros dos patches salvos em {pickle_file}")

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
with open("randomico_melhorado_identificador_por_imgname.pkl", 'rb') as f:
        lista_centro_dict  = pickle.load(f)

In [ ]:
len(lista_centro_dict)

10151

### Geração dos centros pelo método de Segmentação

In [4]:
import os
from torchvision.datasets import ImageFolder

class CustomImageFolder(ImageFolder):
    def __getitem__(self, index):
        img, target = super().__getitem__(index)

        path = self.imgs[index][0]

        image_name = os.path.basename(path)

        return img, target, image_name

In [5]:
img_pretas = []

In [7]:

def generate_patch_centers(image_height, image_width, patch_size):

  stride = patch_size

  num_patches_h = image_height // stride
  num_patches_w = image_width // stride

  centers_h = []
  centers_w = []

  for i in range(num_patches_h):
    centers_h.append((i * stride + stride // 2))

  for j in range(num_patches_w):
    centers_w.append((j * stride + stride // 2))

  centers = []

  for h in centers_h:
    for w in centers_w:
      centers.append((h,w))

  return centers



def remover_fundo_com_grabcut_recortado(imagem):
    mascara = np.zeros(imagem.shape[:2], np.uint8)
    backgroundModel = np.zeros((1, 65), np.float64)
    foregroundModel = np.zeros((1, 65), np.float64)
    altura, largura = imagem.shape[:2]

    x1 = 0
    y1 = 0
    x2 = largura - 1
    y2 = altura - 1

    rectangle = (x1, y1, x2 - x1, y2 - y1)

    cv2.grabCut(imagem, mascara, rectangle,
            backgroundModel, foregroundModel,
            3, cv2.GC_INIT_WITH_RECT)

    mascara_objeto = np.where((mascara == 2) | (mascara == 0), 0, 1).astype('uint8')

    imagem_sem_fundo = imagem * mascara_objeto[:, :, np.newaxis]

    img_recortada = imagem_sem_fundo[y1:y2, x1:x2]

    imagem_gray = cv2.cvtColor(img_recortada, cv2.COLOR_BGR2GRAY)

    _, mascara = cv2.threshold(imagem_gray, 10, 255, cv2.THRESH_BINARY)

    img_original_recortada = imagem[y1:y2, x1:x2]

    return img_recortada, mascara, img_original_recortada












def is_image_black_percentage(image, threshold=0.9):
    total_pixels = image.size

    if image.ndim == 2:
        black_pixels = np.sum(image == 0)
    elif image.ndim == 3:
        black_pixels = np.sum(np.all(image == 0, axis=-1))

    black_percentage = black_pixels / total_pixels
    return black_percentage >= threshold


def validar_centros(centros, patch_size, tamanho_img=(224,224)):
    offset = patch_size // 2
    centros_validos = []

    for (x, y) in centros:
      if (x - offset >= 0 and x + offset - 1 < tamanho_img[0]) and (y - offset >= 0 and y + offset - 1 < tamanho_img[1]):
          centros_validos.append((x, y))

    return centros_validos







def grabcutextractcenters(imagem_tensor, tamanho_img=(224, 224), stride=16, image_name="None"):



  imagem = imagem_tensor.permute(1, 2, 0).cpu().numpy()


  qtd_patches = int((tamanho_img[0]/stride) * (tamanho_img[0]/stride))

  # print(f'Quantidade de patches: {qtd_patches}')

  if imagem.max() <= 1:
    imagem = (imagem * 255).astype(np.uint8)


  imagem = cv2.cvtColor(imagem, cv2.COLOR_RGB2BGR)


  _ , mask, _ = remover_fundo_com_grabcut_recortado(imagem=imagem)

  if is_image_black_percentage(mask):

    altura, largura, _ = imagem.shape

    centers_merge = generate_patch_centers(altura, largura, stride)


    img_pretas.append(image_name)

    return centers_merge
  else:
    mask = cv2.resize(mask, tamanho_img)

    centers_randomicos = []
    centers_randomicos_validos = []
    centers_stride = []
    centers_stride_validos = []
    centers_merge = []
    altura, largura = mask.shape

    for i in range(altura):
        for j in range(largura):
            pixel = mask[i, j]

            if len(pixel.shape) == 0:
                if pixel == 255:
                      centers_randomicos.append((i, j))
            else:
                if np.array_equal(pixel, [255, 255, 255]):
                      centers_randomicos.append((i, j))


    centers_randomicos_validos = validar_centros(centros=centers_randomicos, patch_size=stride)



    for i in range(0, altura, stride):
          for j in range(0, largura, stride):
              pixel = mask[i, j]

              if len(pixel.shape) == 0:
                  if pixel == 255:
                      centers_stride.append((i, j))
              else:
                  if np.array_equal(pixel, [255, 255, 255]):
                      centers_stride.append((i, j))


    centers_stride_validos = validar_centros(centros=centers_stride, patch_size=stride)


    quantidade_patches_stride = len(centers_stride_validos)

    if quantidade_patches_stride < qtd_patches:
      diferentes_lista1 = set(centers_randomicos_validos) - set(centers_stride_validos)
      diferentes_lista2 = set(centers_stride_validos) - set(centers_randomicos_validos)

      resultado = list(diferentes_lista1) + list(diferentes_lista2)

      random.shuffle(resultado)
      qtd_faltante_patches = qtd_patches - quantidade_patches_stride

      centers_merge = centers_stride_validos.copy()
      centers_merge.extend(resultado[0:qtd_faltante_patches])
    elif quantidade_patches_stride == qtd_patches:
      centers_merge = centers_stride_validos.copy()




    # fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # img_original_recortada = cv2.resize(imagem, tamanho_img)

    # axes[0].imshow(mask, cmap='gray')
    # axes[0].set_title('Máscara')
    # axes[0].axis('off')

    # axes[1].imshow(img_original_recortada)
    # axes[1].set_title('Imagem Original Recortada')
    # axes[1].axis('off')

    return centers_merge

In [ ]:
from torchvision.transforms import v2
import pickle
import numpy as np

pickle_file = "segmentacao_dicionario.pkl"

img_size = (224, 224)
batch_size = 32

transform = v2.Compose([
    v2.Resize(img_size),
    v2.ToTensor(),
])


dataset = CustomImageFolder(root=diretorio_destino, transform=transform)

centros_dict = {}

for img_idx in tqdm(range(len(dataset)), desc="Processando imagens"):
    image, label, image_name = dataset[img_idx] 


    centers = grabcutextractcenters(image, stride=32, image_name=image_name)


    centros_dict[image_name] = centers


with open(pickle_file, 'wb') as f:
    pickle.dump(centros_dict, f)


print(f"Centros dos patches salvos em {pickle_file}")

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(
Processando imagens: 100%|██████████| 10151/10151 [2:55:38<00:00,  1.04s/it]


Centros dos patches salvos no google drive em /content/drive/My Drive/segmentacao_dicionario.pkl
Centros dos patches salvos em segmentacao_dicionario.pkl


In [ ]:
with open("segmentacao_dicionario.pkl", 'rb') as f:
        lista_centro_dict  = pickle.load(f)

In [ ]:
len(lista_centro_dict)

10151

In [ ]:
for k, v in lista_centro_dict.items():
  print(k, len(v))

A saída de streaming foi truncada nas últimas 5000 linhas.
5198.png 196
5199.png 196
5200.png 196
5201.png 196
5214.png 196
5220.png 196
5221.png 196
5222.png 196
5223.png 196
5224.png 196
5225.png 196
5226.png 196
5227.png 196
5228.png 196
5229.png 196
5230.png 196
5231.png 196
5232.png 196
5233.png 196
5234.png 196
5235.png 196
5236.png 196
5237.png 196
5238.png 196
5239.png 196
5240.png 196
5241.png 196
5242.png 196
5243.png 196
5244.png 196
5245.png 196
5246.png 196
5247.png 196
5248.png 196
5249.png 196
5250.png 196
5251.png 196
5252.png 196
5253.png 196
5254.png 196
5255.png 196
5256.png 196
5257.png 196
5258.png 196
5259.png 196
5260.png 196
5264.png 196
5265.png 196
5266.png 196
527.png 196
5274.png 196
528.png 196
5285.png 196
5286.png 196
5287.png 196
5288.png 196
5289.png 196
529.png 196
5297.png 196
5298.png 196
5299.png 196
5300.png 196
5301.png 196
5302.png 196
5303.png 196
5308.png 196
5309.png 196
5310.png 196
5317.png 196
5318.png 196
5319.png 196
5320.png 196
5321.png